## Sample task 

Compute matches for July 1, 2019 (361 fires in FEDS input). 

At ~2s/polygon, running one day took 3:00 for the core reference match algorithm. Back of the envelope suggests at this rate it would take about 18h to search one year IF that scales linearly. Loading all polygons for 2018, tqdm estimates it would be 10h, leading to ~50h for the whole dataset I need to process. 

Estimating 24m to run July 1-July 10. 

With 1 worker/1 thread each, 3m 35s to complete full OutputCalculation. 

With 4 workers/1 thread each, 1m 7s to complete full OutputCalculation.

With 8 workers, 53.2s to complete full OutputCalculation. 

Benchmarks run on Apple M2 16GB MacBook Pro. 

In [ ]:
feds_v3_filepath = "/Users/zbecker2/Downloads/allfires_20181231_PM.parq"

# use api data for simplicity 
import Utilities
import Input_FEDS
import Input_Reference
import Output_Calculation

from Input_FEDS import InputFEDS
from Input_Reference import InputReference
from Output_Calculation import OutputCalculation
from Utilities import *

# Start time 
year_start = 2018
month_start = 7
day_start = 1 
hour_start = 0
minute_start = 0
second_start = 0
tz_offset_hours_start = 0
tz_offset_minutes_start = 0
utc_offset_start = '00:00'

# End time 
year_stop = 2018
month_stop = 7
day_stop = 2
hour_stop = 0
minute_stop = 0
second_stop = 0
tz_offset_hours_stop = 0
tz_offset_minutes_stop = 0
utc_offset_stop = '00:00'

crs = 4326

# BBOX FOR SEARCH - [lon, lat, lon, lat]
search_bbox = ["-126.401171875","24.071240929282325","-61.36210937500001","49.40003415463647"]

# DAY SEARCH RANGE- acceptable distance warning to search from feds -> reference (e.g. if reference polygon is 8 days away, output a warning)
# note by default, any results not of the same year are eliminated
day_search_range = 7 


calc_in_parallel = False

# FEDS INPUT SETTINGS

# read local
feds_title = None 
feds_collection = feds_v3_filepath
feds_access_type = "local"
feds_limit = None
feds_filter = False
feds_apply_finalfire = True # only take latest fireID per fire


# REFERENCE INPUT SETTINGS 
ref_title ="Historic_GeoMAC_Perimeters_2018"
ref_control_type = "defined" # or "custom"
ref_custom_url = "none"
ref_custom_read_type = "none"
ref_filter = False # False or a valid query

# OUTPUT SETTINGS
print_on = False
plot_on = False
name_for_output_file = "test_run" 
output_format="csv" 
user_path = "./output"
output_maap_url = f"{user_path}/{name_for_output_file}.{output_format}"

# start date formatting
search_start = Utilities.format_datetime(year_start, 
                                         month_start, 
                                         day_start, 
                                         hour_start, 
                                         minute_start, 
                                         second_start, 
                                         tz_offset_hours_start, 
                                         tz_offset_minutes_start,
                                         utc_offset_start)
# stop date formatting
search_stop = Utilities.format_datetime(year_stop, 
                                        month_stop, 
                                        day_stop, 
                                        hour_stop, 
                                        minute_stop, 
                                        second_stop, 
                                        tz_offset_hours_stop, 
                                        tz_offset_minutes_stop,
                                        utc_offset_stop)

# bound check the bbox
assert Utilities.check_bbox(search_bbox), f"ERR: passed bbox {search_bbox} is not valid; check bounds"
assert  Utilities.check_crs(crs), f"ERR: invalid crs provided {crs}; please enter valid ESPG CRS number"



In [ ]:
%%time 
feds_firenrt = InputFEDS(
                 feds_title, 
                 feds_collection, 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 feds_access_type,
                 feds_limit,
                 feds_filter,
                 feds_apply_finalfire
                )

feds_firenrt

In [ ]:
%%time

nifc_search = InputReference( 
                 search_start,
                 search_stop,
                 search_bbox,
                 crs,
                 ref_title,
                 ref_control_type,
                 ref_custom_url,
                 ref_custom_read_type,
                 ref_filter,
                )

In [ ]:
%%time

my_output = OutputCalculation(
                feds_firenrt,
                nifc_search,
                output_format, 
                output_maap_url,
                day_search_range,
                print_on,
                plot_on
                )